In [1]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 设置matplotlib参数
plt.style.use('default')  # 使用默认样式

# 尝试设置中文字体支持(如果失败则使用默认字体)
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
except:
    print("注意: 中文字体设置失败,将使用默认字体")
    pass


from script.data_structures import *
from script.data_types import *
from script.common import get_raw_data, parse_filename, expand_acc_data, expand_ppg_data, process_and_merge_sensor_data

from script.wear_detection_dll import *


In [2]:
def plot_wear_detection_results(directory, file_name_without_ext, 
                                 expanded_ppg_ir_df=None, 
                                 expanded_ppg_g_df=None, 
                                 expanded_ppg_r_df=None,
                                 expanded_acc_df=None, 
                                 wear_df=None, 
                                 is_show=False, 
                                 result_type='original'):
    """
    绘制佩戴检测结果数据,包括展开的PPG IR、PPG Green、PPG Red、加速度数据和佩戴标志。
    根据可用数据动态调整子图数量和布局。
    
    参数:
        directory: 保存目录
        file_name_without_ext: 文件名(不含扩展名)
        expanded_ppg_ir_df: PPG IR数据(可选)
        expanded_ppg_g_df: PPG Green数据(可选)
        expanded_ppg_r_df: PPG Red数据(可选)
        expanded_acc_df: 加速度数据(可选)
        wear_df: 佩戴信息数据(可选)
        is_show: 是否显示图形
        result_type: 结果类型('original' 或 'simulated')
    """
    # 检查每个数据源是否有效
    has_ppg_ir = expanded_ppg_ir_df is not None and not expanded_ppg_ir_df.empty
    has_ppg_g = expanded_ppg_g_df is not None and not expanded_ppg_g_df.empty
    has_ppg_r = expanded_ppg_r_df is not None and not expanded_ppg_r_df.empty
    has_acc = expanded_acc_df is not None and not expanded_acc_df.empty
    has_wear = wear_df is not None and not wear_df.empty

    # 计算需要的子图数量(PPG图作为一个子图)
    plot_count = sum([has_ppg_ir or has_ppg_g or has_ppg_r, has_acc, has_wear])

    if plot_count == 0:
        print("没有可用的数据进行绘制")
        return

    fig, axes = plt.subplots(plot_count, 1, figsize=(20, 4 * plot_count), sharex=True)
    if plot_count == 1:
        axes = [axes]

    result_label = "原始结果 (Original)" if result_type == 'original' else "模拟结果 (Simulated)"
    fig.suptitle(f"{file_name_without_ext} - {result_label}", fontsize=16, fontweight='bold')
    current_ax_idx = 0

    # 绘制 PPG 数据(IR / Green / Red)
    if has_ppg_ir or has_ppg_g or has_ppg_r:
        ax = axes[current_ax_idx]

        # 为了清晰,使用双 Y 轴：左轴绘制 IR 和 Red,右轴绘制 Green(如果存在)
        ax_twinx = None
        # if has_ppg_g:


        if has_ppg_ir:
            ax.plot(expanded_ppg_ir_df['unix_timestamp'], expanded_ppg_ir_df['ppg_value'], label='PPG IR', color='pink', linewidth=1) # type: ignore
        if has_ppg_r:
            ax.plot(expanded_ppg_r_df['unix_timestamp'], expanded_ppg_r_df['ppg_value'], label='PPG Red', color='red', linewidth=1) # type: ignore
        if has_ppg_g:
            ax_twinx = ax.twinx()
            ax_twinx.set_ylabel('PPG Green', fontsize=12)
            # ax_twinx.plot(expanded_ppg_g_df['unix_timestamp'], expanded_ppg_g_df['ppg_value'], '.', markersize=1, label='PPG Green', color='green', linewidth=0.5) # type: ignore
            ax_twinx.plot(expanded_ppg_g_df['unix_timestamp'], expanded_ppg_g_df['ppg_value'],  label='PPG Green', color='green', linewidth=0.5) # type: ignore

        ax.set_title('Expanded PPG Data')
        ax.set_ylabel('PPG (IR/Red)')
        ax.grid(True, alpha=0.3)

        # 合并图例
        handles, labels = ax.get_legend_handles_labels()
        if ax_twinx is not None:
            h2, l2 = ax_twinx.get_legend_handles_labels()
            handles += h2
            labels += l2
        if handles:
            ax.legend(handles, labels, loc='upper left')

        current_ax_idx += 1

    # 绘制加速度数据
    if has_acc:
        ax = axes[current_ax_idx]
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['x'], label='ACC X', color='red', linewidth=0.5) # type: ignore
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['y'], label='ACC Y', color='green', linewidth=0.5) # type: ignore
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['z'], label='ACC Z', color='blue', linewidth=0.5) # type: ignore
        ax.set_ylabel('Acceleration')
        ax.set_title('Expanded Acceleration Data')
        ax.legend()
        ax.grid(True, alpha=0.3)
        current_ax_idx += 1

    # 绘制佩戴标志
    if has_wear:
        ax = axes[current_ax_idx]
        ax.plot(wear_df['unix_timestamp'], wear_df['wear_flag'], marker='o', markersize=3, linewidth=1, label='Wear Flag', color='orange') # type: ignore
        ax.set_ylabel('Wear Flag')
        ax.set_xlabel('Unix Timestamp (s)')
        ax.set_title('Wear Info Flags')
        ax.legend()
        ax.grid(True, alpha=0.3)

    plt.tight_layout()

    suffix = "_original" if result_type == 'original' else "_simulated"
    output_file = f"{directory}/{file_name_without_ext}_wear{suffix}.png"

    # print(f"图形已保存至: {output_file}")
    if is_show:
        plt.show()
    else:
        plt.savefig(output_file, dpi=300)
        plt.close()


In [4]:
# # %matplotlib qt
# %matplotlib inline  



# directory = r'G:\python\rawdata_Wearing\bug'

# # 获取目录下的所有文件
# all_files = os.listdir(directory)

# # 过滤出 .rawdata 文件
# rawdata_files = [f for f in all_files if f.endswith('.rawdata')]

# # 随机打乱文件顺序
# # random.shuffle(rawdata_files)

# dll_path = r"G:/ncs_workspace/application/algorithm_lib/alg_wear/build/bin/libalg_wear.dll"
# mac_baseline = {}



In [5]:
%matplotlib qt
# %matplotlib inline  

directory = r'rawdata/Motion/0108/'
file_name = "Sleep_Nap_zhangz_20260108_165124_ead7.rawdata"


file_path = os.path.join(directory, file_name)

real_path = os.path.realpath(file_path)
file_name = os.path.basename(file_path)
file_name_without_ext, ext = os.path.splitext(file_name)
directory = os.path.dirname(real_path)
info =  parse_filename(file_name)
mac = info.get('mac','0000').lower()

print(f"real_path: {real_path} \n{directory} {file_name_without_ext}{ext}  MAC:{mac} ")

result = get_raw_data(real_path)

# 获取解析结果,如果不存在则创建空 DataFrame
base_info_df = result.get('base_info_df', pd.DataFrame())
ppg_g_df = result.get('ppg_g_df', pd.DataFrame(columns=['unix_timestamp', 'serial_number', 'ms', 'data','ppg_data']))
ppg_r_df = result.get('ppg_r_df', pd.DataFrame(columns=['unix_timestamp', 'serial_number', 'ms', 'data','ppg_data']))
ppg_ir_df = result.get('ppg_ir_df', pd.DataFrame(columns=['unix_timestamp', 'serial_number', 'ms', 'data','ppg_data']))
wear_info_df = result.get('wear_info_df', pd.DataFrame())
acc_df = result.get('acc_df', pd.DataFrame(columns=['unix_timestamp', 'serial_number', 'ms', 'acc_data']))
data_type_dict = result.get('data_types', {})
TYHX_data_df = result.get('TYHX_data_df', pd.DataFrame())
sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
# print(f"文件 {file_path} 解析完成,包含数据类型: ")
print(f"{"\n".join(readable_types)}")
TYHX_data_df.to_csv(f"{directory}/{file_name_without_ext}_TYHX_data.csv", index=False)
# ppg_r_df.to_csv(f"{directory}/{file_name_without_ext}_ppg_r.csv", index=False)
# ppg_ir_df.to_csv(f"{directory}/{file_name_without_ext}_ppg_ir.csv", index=False)
# ppg_g_df.to_csv(f"{directory}/{file_name_without_ext}_ppg_g.csv", index=False)
# acc_df.to_csv(f"{directory}/{file_name_without_ext}_acc.csv", index=False)
# wear_info_df.to_csv(f"{directory}/{file_name_without_ext}_wear_info.csv", index=False)
(combined_ppg_df, 
wear_info_df,
expanded_acc_df,
expanded_ppg_g_df,
expanded_ppg_ir_df,
expanded_ppg_r_df)= process_and_merge_sensor_data(acc_df, ppg_g_df, ppg_ir_df,ppg_r_df, wear_info_df) 
        
# combined_ppg_df.to_csv(f"{directory}/{file_name_without_ext}_combined_ppg.csv", index=False)


real_path: G:\PyScript\rawdata\Motion\0108\Sleep_Nap_zhangz_20260108_165124_ead7.rawdata 
G:\PyScript\rawdata\Motion\0108 Sleep_Nap_zhangz_20260108_165124_ead7.rawdata  MAC:ead7 
00  BASE_INFO             基础信息                   count: 2695 
15  RAW_PPG_GREEN         原始PPG绿光                count: 10670 
17  RAW_PPG_IR            原始PPG红外                count: 10601 
18  RAW_ACC               原始加速度                  count: 21968 
33  AMBIENT_INFO          环境信息                   count: 10668 
69  MOTION_INFO           运动识别                   count: 10988 
71  WEAR_INFO             佩戴信息                   count: 2776 
82  TYHX_AGC_DATA         TYHX AGC数据             count: 10679 
99  CREEK_HR_INFO         CREEK心率信息              count: 10673 
processed_acc_df 发现 713 条重复的 unix_timestamp, 正在合并...
processed_acc_df 合并后记录数: 10274
processed_ppg_g_df 发现 293 条重复的 unix_timestamp, 正在合并...
processed_ppg_g_df 合并后记录数: 10377
processed_ppg_ir_df 发现 293 条重复的 unix_timestamp, 正在合并...
processed_ppg_ir_df 合并后记录数: 

In [20]:
ppg_g_df['unix_timestamp'].diff().plot()

<Axes: >

In [27]:
%matplotlib qt

# 计算并打印 PPG 数据的均值
if not expanded_ppg_ir_df.empty:
    ppg_ir_mean = expanded_ppg_ir_df['ppg_value'].mean()
    print(f"PPG IR 均值: {ppg_ir_mean:.2f}")
else:
    print("PPG IR: 无数据")

if not expanded_ppg_g_df.empty:
    ppg_g_mean = expanded_ppg_g_df['ppg_value'].mean()
    print(f"PPG Green 均值: {ppg_g_mean:.2f}")
else:
    print("PPG Green: 无数据")

if not expanded_ppg_r_df.empty:
    ppg_r_mean = expanded_ppg_r_df['ppg_value'].mean()
    print(f"PPG Red 均值: {ppg_r_mean:.2f}")
else:
    print("PPG Red: 无数据")

print()  # 空行分隔

plot_wear_detection_results(
    directory=directory,
    file_name_without_ext=file_name_without_ext,
    expanded_ppg_ir_df=expanded_ppg_ir_df,
    expanded_ppg_g_df=expanded_ppg_g_df[0::2],
    expanded_ppg_r_df=expanded_ppg_r_df,
    expanded_acc_df=expanded_acc_df,
    wear_df=wear_info_df,
    is_show=True
)


PPG IR 均值: 38857.49
PPG Green 均值: 337510.55
PPG Red: 无数据



In [11]:
def run_wear_detection_algorithm(df, directory,file_name_without_ext,
                                 dll_path,
                                 ir_threshold=12000,green_threshold= 18000 ,ir_baseline=0
                                 ):
    # 1. 创建一个列表用于存储日志
    log_buffer = []

    # 2. 定义回调：只存列表,不写文件
    def memory_logging_callback(message):
        # print(message, end='') 
        log_buffer.append(message) # 存入内存
    
    
    results = []
    try:
        # 使用上下文管理器自动管理DLL资源
        with WearDetectionDLL(dll_path, print_callback=memory_logging_callback) as dll:
            
            # 获取版本
            version = dll.get_version()
            # print(f"alg_wear_new Ver: {version.major}.{version.minor}.{version.patch}")
            
            # 初始化(戒指模式)
            # dll.init_default(is_ring=True)


            is_ring=True
            config = AlgWearInitParament()
            config.ppg_low_ir_threshold = ir_threshold
            config.ppg_green_threshold = green_threshold
            config.ppg_unwear_cnt = 10
            config.is_ring_flag = is_ring
            # config.wear_amb_std_threshold = 0
            # config.acc_quite_time = 300
            # config.green_confusion_threshold = 0
            # config.confirmed_recheck_timer = 60
            config.ppg_low_ir_baseline = ir_baseline
            dll.init_with_config(config)


            for idx, row in df.iterrows():
                
                
                # 1. 准备 ACC
                acc_data = row['acc_data']
                acc_axis = dll.prepare_acc_data(acc_data)
                unix_timestamp = row['unix_timestamp']
                # 2. 准备 PPG
                ppg_g = row['ppg_g_data']
                ppg_ir = row['ppg_ir_data']
                current = row['data'][1]
                # print(f"Processing row {idx}", )
                # print(f"  ACC data: {acc_data}")
                # print(f"  PPG G data: {ppg_g}")
                # print(f"  PPG IR data: {ppg_ir}")

                # 构造 ppg_data 字典
                ppg_config = {
                    'g': ppg_g,
                    'low_ir': ppg_ir, # 使用 IR 数据
                    'channel_num': 1,
                    'g_current': current, # 模拟电流值
                    'fs': 32, 
                }
                ppg_sensors = dll.prepare_ppg_data(ppg_config)

                # 3. 准备 WearingData
                wearing_data = WearingData()
                wearing_data.measure_type = WearMeasureType.WEAR_IR

                # 4. 执行
                result = dll.process(acc_axis=acc_axis, ppg_sensors=ppg_sensors, wearing_data=wearing_data)
                result_dict = dll.result_to_dict(result)
                result_dict['unix_timestamp'] = unix_timestamp
                # 打印简单的结果验证
                # state_str = "WEAR" if result_dict.get('is_wear') else "UNWEAR" # 假设结果里有这个字段
                # print(f"-> Done. Result: {result_dict}")
                
                results.append(result_dict)

            print(f"\nSuccessfully processed {len(results)} rows.")

    
    except Exception as e:
        print(f"Error running wear detection: {e}")

    finally:
        # 4. 【核心】在 finally 块中将内存中的日志一次性写入文件
        # 这样无论是否报错,日志都会被保存
        log_file_path = os.path.join(directory, f"{file_name_without_ext}_log.txt")
        try:
            with open(log_file_path, 'w', encoding='utf-8') as f:
                # 将列表中的字符串拼接后写入,或者逐行写入
                # 注意：如果 message 自带换行符,就不用加 \n
                for msg in log_buffer:
                    f.write(msg) 
            print(f"日志已保存至: {log_file_path}")
        except Exception as io_e:
            print(f"保存日志失败: {io_e}")

            
    return results

In [14]:
ir_baseline = 0
print(f"使用的ir_baseline: {ir_baseline} for mac: {mac}")
dll_path = r"G:/ncs_workspace/application/algorithm_lib/alg_wear/build/bin/libalg_wear.dll"

wear_result = run_wear_detection_algorithm(combined_ppg_df,directory,file_name_without_ext,
                                            dll_path,
                                            ir_threshold=20000,
                                            ir_baseline=ir_baseline)
wear_df = pd.DataFrame(wear_result)

使用的ir_baseline: 0 for mac: 1307

Successfully processed 2260 rows.
日志已保存至: G:\python\rawdata_bug\HeartRate_Static_lk_20251229_001629_1307_log.txt


In [15]:
plot_wear_detection_results(
    directory=directory,
    file_name_without_ext=file_name_without_ext,
    expanded_ppg_ir_df=expanded_ppg_ir_df,
    expanded_ppg_g_df=expanded_ppg_g_df,
    expanded_ppg_r_df=expanded_ppg_r_df,
    expanded_acc_df=expanded_acc_df,
    wear_df=wear_df,
    is_show=True,
    result_type='Simulated'
)